# K Nearest Neighbours

In [2]:
import numpy as np
import scipy as sp

%run basic_model.ipynb

## BallTree

In [83]:
import heapq
from collections import namedtuple

class BasicTree(BasicModel):
    class Node:
        def __init__(
            self,
            observation_indexes,
            pivot_index,
            left,
            right
        ):
            self.observation_indexes = observation_indexes
            self.pivot_index = pivot_index
            self.left = left
            self.right = right
        
        def is_leaf(self):
            return not self.left and not self.right
    
    def __init__(
        self, 
        X, 
        leaf_size=40, 
        metric='minkowski',
        p=2
    ):
        self.X = super().check_and_transform_X(X)
        
        super().check_value_type_and_set(
            'leaf_size',
            leaf_size,
            int
        )
        
        super().check_value_type_and_set(
            'metric',
            metric,
            str
        )
        
        super().check_value_type_and_set(
            'p',
            p,
            int
        )
    
    def dist(self, x, y):
        return sp.spatial.distance.pdist(
            [x, y],
            self.metric,
            p=self.p
        )[0]
    
    @staticmethod
    def arg_median(array):
        if len(array) % 2 == 1:
            return np.where(array == np.median(array))[0][0]
        else:
            l = len(array) // 2 - 1
            
            left = np.partition(array, l)[l]
            
            return np.where(array == left)[0][0]
    
    @staticmethod
    def split_by_pivot(
        node_sample,
        observation_indexes,
        median_sample_index,
        split_dimension
    ):
        left_split_indexes = []
        right_split_indexes = []
        
        median_value = node_sample[
            median_sample_index,
            split_dimension
        ]
        
        for i in range(node_sample.shape[0]):                
            observation_index = observation_indexes[i]
            
            if node_sample[i, split_dimension] <= median_value:
                left_split_indexes.append(observation_index)
            else:
                right_split_indexes.append(observation_index)
                
        return left_split_indexes, right_split_indexes
    
    PrioritizedPointIndex = namedtuple(
        'PrioritizedPointIndex',
        ['dist_to_target', 'point_index']
    )
    
    def process_leaf(
        self,
        k,
        target_point,
        node,
        heap
    ):
        if len(heap) == 0:
            dummy_index = node.observation_indexes[0]
            
            heap.append(BasicTree.PrioritizedPointIndex(
                dist_to_target=-self.dist(
                    self.X[dummy_index, :],
                    target_point
                ),
                point_index=dummy_index
            ))

        for observation_index in node.observation_indexes:
            from_target_to_current_observation = self.dist(
                target_point,
                self.X[observation_index, :]
            )

            from_target_to_heap_worst = -heap[0].dist_to_target

            if from_target_to_current_observation < from_target_to_heap_worst or \
               len(heap) < k:
                heapq.heappush(heap, BasicTree.PrioritizedPointIndex(
                    dist_to_target=-from_target_to_current_observation,
                    point_index=observation_index
                ))
            
                if len(heap) > k:
                    heapq.heappop(heap)

class BallTree(BasicTree):
    class Node(BasicTree.Node):
        def __init__(
            self, 
            observation_indexes,
            pivot_index=None,
            radius=None,
            left=None,
            right=None
        ):
            super().__init__(
                observation_indexes,
                pivot_index,
                left,
                right
            )
            
            self.radius = radius
    
    def __init__(
        self, 
        X, 
        leaf_size=40, 
        metric='minkowski',
        p=2
    ):
        super().__init__(
            X, 
            leaf_size, 
            metric,
            p
        )
        
        observation_indexes = np.arange(X.shape[0])
        
        self.root = self.__construct_ball(
            observation_indexes
        )
    
    @staticmethod
    def most_spreaded_dimensionality(X):
        max_std = X[:, 0].std()
        most_spreaded_dimensionality = 0
        
        for i in range(1, X.shape[1]):
            curr_std = X[:, i].std()
            
            if curr_std > max_std:
                max_std = curr_std
                most_spreaded_dimensionality = i
        
        return most_spreaded_dimensionality
    
    def __construct_ball(self, observation_indexes):        
        node_sample = self.X[observation_indexes, :]
        
        most_spreaded_dim = BallTree.most_spreaded_dimensionality(node_sample)
        
        # find pivot
        median_sample_index = BasicTree.arg_median(
            node_sample[:, most_spreaded_dim]
        )
        
        # calculate ball radius
        radius = -1
        
        median_point = node_sample[median_sample_index, :]
        
        for i in range(node_sample.shape[0]):            
            radius = max(
                radius,
                self.dist(
                    node_sample[i, :], 
                    median_point
                )
            )
            
        if len(observation_indexes) <= self.leaf_size:
            return BallTree.Node(
                observation_indexes=observation_indexes,
                pivot_index=observation_indexes[median_sample_index],
                radius=radius
            )
        
        # split observations by pivot
        left_split_indexes, right_split_indexes = BasicTree.split_by_pivot(
            node_sample,
            observation_indexes,
            median_sample_index,
            most_spreaded_dim
        )
        
        if len(left_split_indexes) < self.leaf_size or \
           len(right_split_indexes) < self.leaf_size:
            return BallTree.Node(
                observation_indexes=observation_indexes,
                pivot_index=observation_indexes[median_sample_index],
                radius=radius
            )
        
        return BallTree.Node(
            observation_indexes=None,
            pivot_index=observation_indexes[median_sample_index],
            radius=radius,
            left=self.__construct_ball(left_split_indexes),
            right=self.__construct_ball(right_split_indexes)
        )
    
    def k_nearest_neighbours(
        self,
        k,
        target_point,
        node=None,
        heap=None
    ):
        node = node or self.root
        if heap is None:
            heap = []
        
        if len(heap) != 0:
            from_target_to_node_center = self.dist(
                target_point,
                self.X[node.pivot_index, :]
            )

            from_target_to_heap_worst = -heap[0].dist_to_target

            if from_target_to_node_center - node.radius >= from_target_to_heap_worst:
                return
        
        if node.is_leaf():
            self.process_leaf(k, target_point, node, heap)
        else:
            self.k_nearest_neighbours(
                k, 
                target_point, 
                node.left, 
                heap
            )
            
            self.k_nearest_neighbours(
                k, 
                target_point, 
                node.right, 
                heap
            )
        
        return heap

## BallTree Testing

In [81]:
from sklearn.datasets import make_classification
import scipy as sp
import unittest
import time

cl_X, cl_y = make_classification(100, 20)
cl_y = cl_y.reshape((100, 1))

class TestBallTree(unittest.TestCase):    
    def test_samples_in_tree(self):
        def check_samples_in_tree(node, memory):
            if node.is_leaf():
                for el_index in node.observation_indexes:
                    memory.add(el_index)
            else:
                check_samples_in_tree(node.left, memory)
                check_samples_in_tree(node.right, memory)
        
        root = BallTree(cl_X).root
        
        memory = set()
        check_samples_in_tree(root, memory)
        
        self.assertEqual(
            memory,
            set(np.arange(cl_X.shape[0]))
        )
    
    def test_leaf_size(self):
        leaf_size = 20
        
        def check_leaf_size(node):            
            if node.is_leaf():
                node_size = len(node.observation_indexes)
                print(node_size)
                
                self.assertGreaterEqual(
                    node_size,
                    leaf_size
                )
            else:
                check_leaf_size(node.left)
                check_leaf_size(node.right)
        
        bt = BallTree(cl_X, leaf_size=leaf_size)
        
        check_leaf_size(bt.root)
        
        self.assertEqual(
            bt.leaf_size,
            leaf_size
        )
        
    def test_metric(self):
        metric = 'chebyshev'
        
        bt = BallTree(cl_X, metric=metric)
        
        self.assertEqual(
            bt.metric,
            metric
        )
    
    def test_dist(self):
        p = 3
        
        bt = BallTree(cl_X, p=p)
        
        self.assertEqual(
            sp.spatial.distance.pdist(
                [[1, 2], [45, 3]],
                bt.metric,
                p=p
            )[0],
            bt.dist([1, 2], [45, 3])
        )

In [84]:
with open('tmp', "w") as f:
    runner = unittest.TextTestRunner(f)
    obj = unittest.main(
        argv=['first-arg-is-ignored', '--verbose', 'TestBallTree'], 
        testRunner=runner,
        exit=False
    )

! cat tmp
! rm -r tmp

25
25
25
25
....
----------------------------------------------------------------------
Ran 4 tests in 0.041s

OK


## KDTree

In [89]:
import heapq

class KDTree(BasicTree):
    class Node(BasicTree.Node):
        def __init__(
            self,
            observation_indexes,
            pivot_index,
            left,
            right
        ):
            super().__init__(
                observation_indexes,
                pivot_index,
                left,
                right
            )
    
    def __init__(
        self, 
        X, 
        leaf_size=40, 
        metric='minkowski',
        p=2
    ):
        super().__init__(
            X, 
            leaf_size, 
            metric,
            p
        )
        
        observation_indexes = np.arange(X.shape[0])
        
        self.root = self.__construct_tree(
            observation_indexes
        )
    
    def __construct_tree(self, observation_indexes, depth=1):
        if len(observation_indexes) <= self.leaf_size:
            return KDTree.Node(
                observation_indexes,
                None,
                None,
                None
            )
        
        split_dimension = depth % self.X.shape[1]
        
        node_sample = self.X[observation_indexes, :]
        
        median_sample_index = BasicTree.arg_median(
            node_sample[:, split_dimension]
        )
        
        left_split_indexes, right_split_indexes = BasicTree.split_by_pivot(
            node_sample,
            observation_indexes,
            median_sample_index,
            split_dimension
        )
        
        if len(left_split_indexes) < self.leaf_size or \
           len(right_split_indexes) < self.leaf_size:
            return KDTree.Node(
                observation_indexes,
                None,
                None,
                None
            )
        
        return KDTree.Node(
            observation_indexes=None,
            pivot_index=observation_indexes[median_sample_index],
            left=self.__construct_tree(left_split_indexes, depth+1),
            right=self.__construct_tree(right_split_indexes, depth+1)
        )
    
    def __calc_dists_in_suitable_leaf(
        self,
        k,
        target_point,
        node,
        heap,
        depth
    ):
        if node.is_leaf():
            self.process_leaf(
                k,
                target_point,
                node,
                heap
            )
        else:
            split_dimension = depth % self.X.shape[1]
            split_value = self.X[node.pivot_index, split_dimension]

            if target_point[split_dimension] <= split_value:
                self.__calc_dists_in_suitable_leaf(
                    k,
                    target_point,
                    node.left,
                    heap,
                    depth+1
                )
            else:
                self.__calc_dists_in_suitable_leaf(
                    k,
                    target_point,
                    node.right,
                    heap,
                    depth+1
                )
                
    def __collect_leaf_nodes_by_mask(
        self,
        node,
        target_point,
        spherical_mask_radius,
        collected_nodes
    ):        
        if node.left.is_leaf() or node.right.is_leaf():
            from_split_node_to_target = self.dist(
                self.X[node.pivot_index, :],
                target_point
            )
            
            if from_split_node_to_target < spherical_mask_radius:
                if node.left.is_leaf():
                    collected_nodes.append(node.left)
                if node.right.is_leaf():
                    collected_nodes.append(node.right)
        
        if not node.left.is_leaf():
            self.__collect_leaf_nodes_by_mask(
                node.left,
                target_point,
                spherical_mask_radius,
                collected_nodes
            )
        
        if not node.right.is_leaf():
            self.__collect_leaf_nodes_by_mask(
                node.right,
                target_point,
                spherical_mask_radius,
                collected_nodes
            )
    
    def k_nearest_neighbours(
        self,
        k,
        target_point
    ):
        heap = []
        
        self.__calc_dists_in_suitable_leaf(
            k,
            target_point,
            self.root,
            heap=heap,
            depth=1
        )
        
        spherical_mask_radius = -heap[0].dist_to_target
        
        intersected_by_mask_nodes = []
        
        self.__collect_leaf_nodes_by_mask(
            self.root,
            target_point,
            spherical_mask_radius,
            intersected_by_mask_nodes
        )
        
        for node in intersected_by_mask_nodes:
            self.process_leaf(
                k,
                target_point,
                node,
                heap
            )
        
        return heap

## KDTree testing

In [91]:
from sklearn.datasets import make_classification
import scipy as sp
import unittest
import time

cl_X, cl_y = make_classification(100, 20)
cl_y = cl_y.reshape((100, 1))

class TestKDTree(unittest.TestCase):    
    def test_samples_num_in_tree(self):
        def count_samples(node):
            if node.is_leaf():
                node_count_samples = len(node.observation_indexes)
                
                return node_count_samples
            
            left_count_samples = count_samples(node.left)
            right_count_samples = count_samples(node.right)
            
            return left_count_samples + right_count_samples
        
        root = KDTree(cl_X).root
        
        self.assertEqual(
            count_samples(root),
            cl_X.shape[0]
        )
    
    def test_leaf_size(self):
        leaf_size = 20
        
        def check_leaf_size(node):
            if node.is_leaf():
                node_size = len(node.observation_indexes)
                
                self.assertGreaterEqual(
                    node_size,
                    leaf_size
                )
            else:
                check_leaf_size(node.left)
                check_leaf_size(node.right)
        
        kd = KDTree(cl_X, leaf_size=leaf_size)
        
        check_leaf_size(kd.root)
        
        self.assertEqual(
            kd.leaf_size,
            leaf_size
        )
        
    def test_metric(self):
        metric = 'chebyshev'
        
        kd = KDTree(cl_X, metric=metric)
        
        self.assertEqual(
            kd.metric,
            metric
        )
    
    def test_dist(self):
        p = 3
        
        kd = KDTree(cl_X, p=p)
        
        self.assertEqual(
            sp.spatial.distance.pdist(
                [[1, 2], [45, 3]],
                kd.metric,
                p=p
            )[0],
            kd.dist([1, 2], [45, 3])
        )

In [92]:
with open('tmp', "w") as f:
    runner = unittest.TextTestRunner(f)
    obj = unittest.main(
        argv=['first-arg-is-ignored', '--verbose', 'TestKDTree'], 
        testRunner=runner,
        exit=False
    )

! cat tmp
! rm -r tmp

....
----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK


# K Nearest Neighbours

In [42]:
import heapq

class KNearestNeighbours(BasicModel):
    def __init__(
        self,
        k_neighbours=5,
        weights='uniform',
        algorithm='ball_tree',
        task='classification',
        leaf_size=30, 
        metric='minkowski',
        p=2
    ):        
        super().check_value_type_and_set(
            'k_neighbours',
            k_neighbours,
            int
        )
        
        if not callable(weights):
            super().check_value_and_set(
                'weights',
                weights,
                ['uniform', 'distance']
            )
        else:
            self.weights = weights
        
        super().check_value_and_set(
            'algorithm',
            algorithm,
            ['ball_tree', 'kd_tree', 'brute']
        )
        
        super().check_value_and_set(
            'task',
            task,
            ['classification', 'regression']
        )
        
        super().check_value_type_and_set(
            'leaf_size',
            leaf_size,
            int
        )
        
        super().check_value_type_and_set(
            'metric',
            metric,
            str
        )
        
        super().check_value_type_and_set(
            'p',
            p,
            int
        )
    
    def fit(self, X, y):
        self.X = super().check_and_transform_X(X)
        self.y = super().check_and_transform_y(X, y)
        
        tree_kwargs = {
            'leaf_size': self.leaf_size,
            'metric': self.metric,
            'p': self.p
        }
        
        if self.algorithm == 'ball_tree':
            self.predictor_tree = BallTree(
                X,
                **tree_kwargs
            )
        elif self.algorithm == 'kd_tree':
            self.predictor_tree = KDTree(
                X,
                **tree_kwargs
            )
        else:
            self.predictor_tree = None
    
    def __get_answer(
        self,
        k_nearest_neighbour_indexes,
        target
    ):
        labels = self.y[k_nearest_neighbour_indexes, :].ravel()
        
        weights = []
        
        if self.weights == 'uniform':
            weights = [1] * self.k_neighbours
        else:
            weights = []
            
            for neighbour_index in k_nearest_neighbour_indexes:
                weights.append(BasicTree.dist(
                    self,
                    target,
                    self.X[neighbour_index, :]
                ))
            
            if callable(self.weights):
                weights = self.weights(weights)
        
        if self.task == 'classification':
            ones = ((labels == 1) * weights).sum()
            zeros = ((labels == 0) * weights).sum()
            
            if ones < zeros:
                return 0
            else:
                return 1
        else:
            return (labels * weights).mean()
    
    def __tree_predict(self, X):
        predictions = []
        
        for target in X:
            k_nearest_neighbours = \
                self.predictor_tree.k_nearest_neighbours(
                    self.k_neighbours,
                    target
                )
            
            k_nearest_neighbours_indexes = \
                [el.point_index for el in k_nearest_neighbours]
            
            predictions.append(
                self.__get_answer(
                    k_nearest_neighbours_indexes,
                    target
                )
            )
        
        return predictions
    
    def __brute_force(self, X):
        predictions = []
        
        for target in X:
            heap = []
            
            for predictor_index in range(self.X.shape[0]):
                dist_to_target = BasicTree.dist(
                    self,
                    target,
                    self.X[predictor_index, :]
                )
                
                heapq.heappush(heap, BasicTree.PrioritizedPointIndex(
                    dist_to_target=-dist_to_target,
                    point_index=predictor_index
                ))
                
                if len(heap) > self.k_neighbours:
                    heapq.heappop(heap)
            
            k_nearest_neighbours_indexes = \
                [el.point_index for el in heap]
            
            predictions.append(
                self.__get_answer(
                    k_nearest_neighbours_indexes,
                    target
                ),
            )
        
        return predictions
        
    def predict(self, X):
        X = super().check_and_transform_X(X)
        
        predictions = None
        
        if self.predictor_tree:
            predictions = self.__tree_predict(X)
        else:
            predictions = self.__brute_force(X)
            
        return np.array(predictions).reshape((-1, 1))

# Testing

In [100]:
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error
import unittest
import time

cl_X, cl_y = make_classification(100, 20)
cl_y = cl_y.reshape((100, 1))

regr_X, regr_y = make_regression(100, 20)
regr_y = regr_y.reshape((100, 1))

def time_fit_predict(
    X, 
    y,
    name='BallTree',
    score_names=['ROC AUC'],
    score_funcs=[roc_auc_score],
    *args,
    **kwargs
):
    start = time.time()
    
    knn = KNearestNeighbours(*args, **kwargs)
    knn.fit(X, y)
    
    for score_name, score_func in zip(score_names, score_funcs):
        score = score_func(cl_y, knn.predict(cl_X))

        print("{} {} score: {}".format(
            name,
            score_name,
            score
        ))
        print(args),
        print(kwargs)
    print("Time: {}\n\n".format(time.time() - start))
    
class TestKNearestNeighbours(unittest.TestCase):
    def test_k_neighbours(self):
        k_values = [3, 5, 7]
        
        for k in k_values:
            time_fit_predict(
                cl_X, 
                cl_y, 
                k_neighbours=k
            )
    def test_weights(self):
        weights_values = [
            'uniform', 
            'distance',
            lambda _: 1
        ]
        
        print('First and third scores should be equal')
        
        for weights in weights_values:
            time_fit_predict(
                cl_X, 
                cl_y, 
                weights=weights
            )
    
    def test_ball_tree(self):
        time_fit_predict(
            cl_X,
            cl_y,
            name='BallTree',
            algorithm='ball_tree'
        )
        
    def test_kd_tree(self):
        time_fit_predict(
            cl_X,
            cl_y,
            name='KDTree',
            algorithm='kd_tree'
        )
    
    def test_brute(self):
        time_fit_predict(
            cl_X,
            cl_y,
            name='Brute',
            algorithm='brute'
        )
    
    def test_leaf_size(self):
        leaf_size = 20
        
        knn = KNearestNeighbours(leaf_size=leaf_size)
        knn.fit(cl_X, cl_y)
        
        self.assertEqual(
            knn.predictor_tree.leaf_size,
            knn.leaf_size
        )
        
        self.assertEqual(
            leaf_size,
            knn.leaf_size
        )
    
    def test_p(self):
        p = 4
        
        knn = KNearestNeighbours(p=p)
        knn.fit(cl_X, cl_y)
        
        self.assertEqual(
            knn.predictor_tree.p,
            knn.p
        )
        
        self.assertEqual(
            p,
            knn.p
        )
        
    def test_metric(self):
        metric = 'chebyshev'
        
        knn = KNearestNeighbours(metric=metric)
        knn.fit(cl_X, cl_y)
        
        self.assertEqual(
            knn.predictor_tree.metric,
            knn.metric
        )
        
        self.assertEqual(
            metric,
            knn.metric
        )
    
    def test_regression(self):
        k_values = [3, 5, 7]
        
        for k in k_values:
            time_fit_predict(
                regr_X,
                regr_y,
                task='regression',
                k_neighbours=k,
                score_names=['MSE', 'MAE'],
                score_funcs=[
                    mean_squared_error,
                    mean_absolute_error
                ],
            )

In [101]:
with open('tmp', "w") as f:
    runner = unittest.TextTestRunner(f)
    obj = unittest.main(
        argv=['first-arg-is-ignored', '--verbose', 'TestKNearestNeighbours'], 
        testRunner=runner,
        exit=False
    )

! cat tmp
! rm -r tmp

BallTree ROC AUC score: 0.82
()
{'algorithm': 'ball_tree'}
Time: 0.21013712882995605


Brute ROC AUC score: 0.82
()
{'algorithm': 'brute'}
Time: 0.21593809127807617


BallTree ROC AUC score: 0.86
()
{'k_neighbours': 3}
Time: 0.21288037300109863


BallTree ROC AUC score: 0.82
()
{'k_neighbours': 5}
Time: 0.22194766998291016


BallTree ROC AUC score: 0.8699999999999999
()
{'k_neighbours': 7}
Time: 0.22185969352722168


KDTree ROC AUC score: 0.88
()
{'algorithm': 'kd_tree'}
Time: 0.13137006759643555


BallTree MSE score: 4093.8122812872134
()
{'task': 'regression', 'k_neighbours': 3}
BallTree MAE score: 50.20275938393599
()
{'task': 'regression', 'k_neighbours': 3}
Time: 0.4266963005065918


BallTree MSE score: 4016.5215834545775
()
{'task': 'regression', 'k_neighbours': 5}
BallTree MAE score: 50.781904477399564
()
{'task': 'regression', 'k_neighbours': 5}
Time: 0.414050817489624


BallTree MSE score: 3975.569193260506
()
{'task': 'regression', 'k_neighbours': 7}
BallTree MAE score: 53.34